In [ ]:
''' 搭建简单的搜索引擎 '''
''' 一个搜索引擎由：搜索器query、索引器index、检索器search和用户接口user interface四个部分组成 '''

In [70]:
import os
import re

# base
class SearchEngineBase(object):
    def __init__(self):
        pass
    
    def add_corpus(self, file_path):
        with open(file_path, 'r') as fin:
            text = fin.read()
            self.process_corpus(file_path, text)
    
    
    def process_corpus(self, id, text):
        raise Exception('process_corpus not implemented.')  
    
    def search(self, query):
        raise Exception('search not implemented.')

def main(search_engine):
    for file_path in os.listdir('data'):
        if re.search('.txt', file_path):
            search_engine.add_corpus('data/'+file_path)
        
    while True: 
        query = input()
        if query == 'Q' or query == 'q':
            break
        results = search_engine.search(query) 
        print('found {} result(s):'.format(len(results))) 
        for result in results: 
            print(result)

In [71]:
#构造最小堆 
class MinHeap():
    def __init__(self, maxSize=None):
        self.maxSize = maxSize 
        self.array = [None] * maxSize 
        self._count = 0
    
    def length(self):
        return self._count 
    
    def show(self):
        if self._count <= 0:
            print('null')
        print(self.array[: self._count], end=', ')
    
    def add(self, value):
        #增加元素
        if self._count >= self.maxSize:
            raise Exception('The array is Full')
        self.array[self._count] = value
        self._shift_up(self._count)
        self._count += 1
    
    def _shift_up(self, index):
        #比较结点与根节点的大小， 较小的为根结点
        if index > 0:
            parent = (index - 1) // 2
            if self.array[parent] > self.array[index]:
                self.array[parent], self.array[index] = self.array[index], self.array[parent]
                self._shift_up(parent)
    
    def extract(self):           
        #获取最小值，并更新数组
        if self._count <= 0:
            raise Exception('The array is Empty')
        value = self.array[0]
        self._count -= 1 #更新数组的长度
        self.array[0] = self.array[self._count] #将最后一个结点放在前面
        self._shift_down(0)
        
        return value 
             
    def _shift_down(self, index):  
        #此时index 是根结点
        if index < self._count:
            left = 2 * index + 1
            right = 2 * index + 2
            #判断左右结点是否越界，是否小于根结点，如果是这交换
            if left < self._count and right < self._count and self.array[left] < self.array[index] and self.array[left] < self.array[right]:
                self.array[index], self.array[left] = self.array[left], self.array[index] #交换得到较小的值
                self._shift_down(left)
            elif left < self._count and right < self._count and self.array[right] < self.array[left] and self.array[right] < self.array[index]:
                self.array[right], self.array[index] = self.array[index], self.array[right]
                self._shift_down(right)
            
            #特殊情况： 如果只有做叶子结点
            if left < self._count and right > self._count and self.array[left] < self.array[index]:
                self.array[left], self.array[index] = self.array[index], self.array[left]


In [95]:
import pylru

class LRUCache(object): 
    def __init__(self, size=32): 
        self.cache = pylru.lrucache(size) 
    
    def has(self, key): 
        return key in self.cache 
    
    def get(self, key): 
        return self.cache[key] 
    
    def set(self, key, value): 
        self.cache[key] = value

In [96]:
class BOWInvertedIndexEngine(SearchEngineBase, LRUCache):
    def __init__(self):
        super(BOWInvertedIndexEngine, self).__init__()
        LRUCache.__init__(self)
        self.inverted_index = {}
        self.copy_inverted_index = {}
    
    def process_corpus(self, id, text):
        words = self.parse_text2words(text)
        id_num = int(re.sub('[^\d ]','',id))
        for word in words:
            if word not in self.inverted_index:
                self.inverted_index[word] = MinHeap(10)
            self.inverted_index[word].add(id_num)
    
    def search(self, query):
        if self.has(query): 
            print('cache hit!') 
            return self.get(query)
        
        import copy
        # 深度拷贝，要不然heap会被弄掉
        self.copy_inverted_index = copy.deepcopy(self.inverted_index)
        query_words = self.parse_text2words(query)
        # 用于记录每个word当前索引文档id的位置，超出了倒排记录表长度，则退出
        query_words_cur_index = [0 for query_word in query_words]
        
        for query_word in query_words:
            if query_word not in self.copy_inverted_index:
                return []
        
        result = []
        # 用于比较每个word的当前最小的id, 初始化
        cur_vals = [-1 for query_word in query_words]
        next_flags =[1 for query_word in query_words]
        while True:
            for idx, query_word in enumerate(query_words):
                # 当前word的id不需要往下走
                if next_flags[idx]==0:
                    continue
                if self.copy_inverted_index[query_word].length()<query_words_cur_index[idx]:
                    # 设置cache
                    self.set(query, result)
                    return result
                # 记录min_vals
                cur_vals[idx] = self.copy_inverted_index[query_word].extract()
                
                query_words_cur_index[idx] += 1
            
            max_val = max(cur_vals)
            count = sum(list(map(lambda x: 1 if x==max_val else 0, cur_vals)))
            next_flags = list(map(lambda x: 0 if x==max_val else 1, cur_vals))
            if count == len(query_words):
                result.append(max_val)
                next_flags = [1 for query_word in query_words]
        
                
    @staticmethod
    def parse_text2words(text):
        # 使用正则表达式去除标点符号和换行符
        text = re.sub(r'[^\w ]', ' ', text) # sub 是去掉
        text = text.lower()
        word_list = text.split(' ')
        # 去除空白单词
        word_list = filter(None, word_list)
        return set(word_list)

In [98]:
search_engine = BOWInvertedIndexEngine()

main(search_engine)

song
found 2 result(s):
1
3
song
cache hit!
found 2 result(s):
1
3
123
found 2 result(s):
1
2
Q
